In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import linear_model as linMod
import toolbox as tb;
import kaggleData as kD;

We generate toydata

In [ ]:
#toydata shall have n vectors with 5 dimensions
n = 100000
#probability for signal-label
s_prob = 0.05
dim = 4
data = tb.createToyData(n,dim,s_prob)
weights = data[:,0]
labels = data[:,1]
x_1 = data[:,2]
x_2 = data[:,3]

visualize

In [ ]:
%pylab inline
plt.scatter(x_1, x_2, edgecolor="", c=labels, alpha=0.5)

split toydata into training- and testset for the classifier

In [ ]:
n_train = int(n/10)

train_x_1,test_x_1 = tb.splitList(x_1,n_train)
train_x_2,test_x_2 = tb.splitList(x_2,n_train)
train_labels,test_labels = tb.splitList(labels,n_train)
test_weights = tb.splitList(weights,n_train)[1]

For Comparison, we calculate the best possible AMS    
(case: every signal correctly detected)

In [ ]:
tb.calcMaxAMS(test_weights,test_labels);

we initialize the Logistic Regression Classifier, shape the input-data and fit the model

In [ ]:
logReg = linMod.LogisticRegression(C=1e5)

train_x = np.array([train_x_1,train_x_2]).transpose()
test_x = np.array([test_x_1,test_x_2]).transpose()
train_labels = np.array(train_labels).transpose()
test_labels = np.array(test_labels).transpose()

logReg.fit(train_x,train_labels)

logReg.sparsify()

predProb = logReg.predict_proba(test_x)
pred = logReg.predict(test_x)
score = logReg.score(test_x,test_labels)

print("Score:", score)

In [ ]:
s,b = tb.calcWeightSums(test_weights,pred,test_labels)
print("AMS:",tb.calcAMS(s,b))

We successfully tested logistic Regression, now let's use it on actual CERN-Data.           

In [21]:
(header,
 test_data,
 test_weights,
 test_labels,
 train_data,
 train_weights,
 train_labels) = kD.getWholeDataSet(kSet=["b","v"])

Trainingset has key "t"                   
Public Testset has key "b"   
Private Testset has key "v"

In [22]:
train_eventList,test_eventList = kD.getFeatureSets("EventId",header,test_data,train_data)

How are labels and weights related?

In [ ]:
signal_sum = int(test_labels.cumsum()[-1])
background_sum = int(len(test_labels)-signal_sum)
signal_weight = 0
background_weight = 0
for i in range(0,len(test_labels)):
    if test_labels[i] > 0:
        signal_weight += test_weights[i]
    else:
        background_weight += test_weights[i]
print("Mean of background-weights:", background_weight/background_sum)
print("Mean of signal-weights:",signal_weight/signal_sum)

We observe that False signals are weighted a lot heavier than True signals. 

If a classifier achieved a higher AMS while detecting less signals,   
we can make statements about the usabilty of the features, the classifier used.

We choose features with beneficial properties for classifying.

In [ ]:
(train_DER_met_phi_centrality,
 test_DER_met_phi_centrality) = kD.getFeatureSets("DER_met_phi_centrality",header,test_data,train_data)
(train_DER_pt_ratio_lep_tau,
 test_DER_pt_ratio_lep_tau) = kD.getFeatureSets("DER_pt_ratio_lep_tau",header,test_data,train_data)

Using DER_mass_MMC was not allowed in the former contest, we use it here anyway to test our classifier

In [ ]:
(train_DER_mass_MMC,
 test_DER_mass_MMC) = kD.getFeatureSets("DER_mass_MMC",header,test_data,train_data)

In [ ]:
train_labels = np.array(train_labels).transpose()
test_labels = np.array(test_labels).transpose()

In [ ]:
tb.calcMaxAMS(test_weights,test_labels);

We start with one feature and add more with every regression to see improvement of the AMS

In [4]:
def logisticReg(train_x,train_labels,test_x,test_labels):
    logReg = None
    logReg = linMod.LogisticRegression()
    logReg.fit(train_x,train_labels)
    logReg.sparsify()
    predProb = logReg.predict_proba(test_x)
    pred = logReg.predict(test_x)
    signals = int(pred.cumsum()[-1])  
    return predProb,pred

In [5]:
def logRegFor(fList):
    for feature in fList:
        print("Feature:",feature)
        trainList_x,testList_x = kD.getFeatureSets(feature)
        train_x = np.array([trainList_x]).transpose()
        test_x = np.array([testList_x]).transpose()
        logisticReg(train_x,train_labels,test_x,test_labels)[1];

In [31]:
(train_PRI_tau_pt,
 test_PRI_tau_pt) = kD.getFeatureSets("PRI_tau_pt",header,test_data,train_data)
(train_DER_met_phi_centrality,
 test_DER_met_phi_centrality) = kD.getFeatureSets("DER_met_phi_centrality",header,test_data,train_data)
(train_DER_pt_h,
 test_DER_pt_h) = kD.getFeatureSets("DER_pt_h",header,test_data,train_data)
(train_DER_pt_ratio_lep_tau,
 test_DER_pt_ratio_lep_tau) = kD.getFeatureSets("DER_pt_ratio_lep_tau",header,test_data,train_data)
(train_DER_mass_transverse_met_lep,
 test_DER_mass_transverse_met_lep) = kD.getFeatureSets("DER_mass_transverse_met_lep",header,test_data,train_data)

We are able to achieve a higher AMS by adjusting the decision-threshold

In [6]:
def customThreshold(pred,t = 0.5):
    newPred = np.zeros(len(pred))
    for i in range(0,len(pred)):
        if pred[i] > t:
            newPred[i]=1
    return newPred   

In [7]:
def bestThreshold(predProb,test_weights,test_labels):
    bestPred = predProb
    thresh = 0
    maxAMS = 0
    maxThresh = 0
    newSignals = 0
    for thresh in np.linspace(1.0,0.0,100):
        newPred = customThreshold(predProb,thresh)
        s,b = tb.calcWeightSums(test_weights,newPred,test_labels)
        #print("s:",s,"b:",b)
        ams = tb.calcAMS(s,b)
        #print("ams:",ams)
        if ams > maxAMS:
            bestPred = newPred
            maxThresh = thresh
            maxAMS = ams
            newSignals = int(newPred.cumsum()[-1])
    #print("Maximum AMS:",maxAMS, "with threshold", maxThresh)
    #print("Signals read:", newSignals)
    return bestPred,maxAMS,maxThresh,newSignals

In [8]:
def compareBinaryArrays(a,b):
    if np.shape(a) != np.shape(b):
        print("ERROR: Arrays must have same shape.")
        return None
    eq_total = 0
    eq_ones = 0
    eq_zeros = 0
    for i in range(0,len(a)):
        if a[i] == b[i]:
            eq_total += 1
            if a[i] == 1:
                eq_ones += 1
            else:
                eq_zeros += 1
    return eq_total,eq_ones,eq_zeros

In [9]:
def fullEvaluation(predProb,pred,test_weights,test_labels):
    signals = pred.cumsum()[-1]
    correct = np.equal(pred,test_labels).cumsum()[-1]    
    s,b = tb.calcWeightSums(test_weights,pred,test_labels)
    ams = tb.calcAMS(s,b)
   
    newPred,maxAMS,maxThresh,newSignals = bestThreshold(predProb[:,1],test_weights,test_labels)
    newcorrect = np.equal(newPred,test_labels).cumsum()[-1]
    
    T_eq1,T_eq0 = compareBinaryArrays(test_labels,test_labels)[1:]
    o_eqt,o_eq1,o_eq0 = compareBinaryArrays(pred,test_labels)
    a_eqt,a_eq1,a_eq0 = compareBinaryArrays(newPred,test_labels)
    
    print("Signals in test-data:", test_labels.cumsum()[-1])
    print("Comparison of [o]riginal and [a]djusted predictions:\n"+
          " - [o]Signals read:", signals,"\n"+
          " - [a]Signals read:", newSignals,"\n"+
          " -- Difference:", (signals-newSignals),"\n"+
          " - [o]Correct labels:", o_eqt,"| signals:", o_eq1,"| background:",o_eq0,"\n"+
          " ----- wrong signals:", (T_eq1-o_eq1), "| background:", (T_eq0-o_eq0),"\n"+
          " - [a]Correct labels:", a_eqt,"| signals:", a_eq1,"| background:",a_eq0,"\n"+
          " ----- wrong signals:", (T_eq1-a_eq1), "| background:", (T_eq0-a_eq0),"\n"+
          " -- Difference labels:", (a_eqt-o_eqt),"| signals:", (a_eq1-o_eq1),"| background:",(a_eq0-o_eq0),"\n"+
          " - [o]AMS:", ams,"\n"+
          " - [a]AMS:", maxAMS,"(threshold =",maxThresh,")\n"+
          " -- Difference:", (ams-maxAMS),"\n"
         )
    return pred,newPred

In [36]:
train_X = np.array(
    [train_PRI_tau_pt,
     train_DER_met_phi_centrality,
     train_DER_pt_h,
     train_DER_pt_ratio_lep_tau]).transpose()
test_X = np.array(
    [test_PRI_tau_pt,
     test_DER_met_phi_centrality,
     test_DER_pt_h,
     test_DER_pt_ratio_lep_tau]).transpose()

In [37]:
logReg = linMod.LogisticRegression()
logReg.fit(train_X,train_labels)
logReg.sparsify()
predProb = logReg.predict_proba(test_X)
pred = logReg.predict(test_X)
signals = int(pred.cumsum()[-1])  

s,b = tb.calcWeightSums(test_weights,pred,test_labels)
ams = tb.calcAMS(s,b)
print("AMS:", ams)

AMS: 1.8693410466451816


In [38]:
(predProb,
 pred) = logisticReg(
    train_X,
    train_labels,
    test_X,
    test_labels);
fullEvaluation(predProb,pred,test_weights,test_labels);

Signals in test-data: 187708.0
Comparison of [o]riginal and [a]djusted predictions:
 - [o]Signals read: 107786.0 
 - [a]Signals read: 343589 
 -- Difference: -235803.0 
 - [o]Correct labels: 385614 | signals: 65554 | background: 320060 
 ----- wrong signals: 122154 | background: 42232 
 - [a]Correct labels: 330731 | signals: 156014 | background: 174717 
 ----- wrong signals: 31694 | background: 187575 
 -- Difference labels: -54883 | signals: 90460 | background: -145343 
 - [o]AMS: 1.8693410466451816 
 - [a]AMS: 2.0063041235163688 (threshold = 0.242424242424 )
 -- Difference: -0.13696307687118714 



In [39]:
#tb.createSolutionFile(test_eventList,predProb[:,1],0.43,"F:\BA_git\Data\Solutions\solution_logReg_test_4.csv")

In [ ]:
print("[o] correct signal/correct labels:",(53744/315491))
print("[o] correct background/correct labels:",(261747/315491))
print("[a] correct signal/correct labels:",(124621/275182))
print("[a] correct background/correct labels:",(150561/275182))
print("")
print("[o] wrong signal/wrong labels:",(99939/(len(test_labels)-315491)))
print("[o] wrong background/wrong labels:",(34570/(len(test_labels)-315491)))
print("[a] wrong signal/wrong labels:",(29062/(len(test_labels)-275182)))
print("[a] wrong background/wrong labels:",(145756/(len(test_labels)-275182)))

By adjusting the threshold, we raise the rate of correct signals at the expense of correctly predicted background-events.
This results in a higher AMS even though we softened our decision-threshold

In [ ]:
train_x = np.array(
    [train_DER_met_phi_centrality,
     train_DER_pt_ratio_lep_tau]).transpose()
test_x = np.array(
    [test_DER_met_phi_centrality,
     test_DER_pt_ratio_lep_tau]).transpose()
predProb,pred = logisticReg(
    train_x,train_labels,
    test_x,
    test_labels);
fullEvaluation(predProb,pred,test_weights,test_labels);

In [23]:
#all but events, pls
train_X = train_data[:,1:]
test_X = test_data[:,1:]

In [29]:
predProb,pred = logisticReg(
    train_X,train_labels,
    test_X,
    test_labels)[0:2];
fullEvaluation(predProb,pred,test_weights,test_labels);

Signals in test-data: 187708.0
Comparison of [o]riginal and [a]djusted predictions:
 - [o]Signals read: 150915.0 
 - [a]Signals read: 191061 
 -- Difference: -40146.0 
 - [o]Correct labels: 412973 | signals: 100798 | background: 312175 
 ----- wrong signals: 86910 | background: 50117 
 - [a]Correct labels: 412675 | signals: 120722 | background: 291953 
 ----- wrong signals: 66986 | background: 70339 
 -- Difference labels: -298 | signals: 19924 | background: -20222 
 - [o]AMS: 2.8303462628376184 
 - [a]AMS: 2.8937862066934925 (threshold = 0.434343434343 )
 -- Difference: -0.06343994385587415 



private rank #1485!

We should use more features, but also get rid of too noisy ones

In [30]:
tb.createSolutionFile(test_eventList,predProb[:,1],0.43,"F:\BA_git\Data\Solutions\solution_logReg_test.csv")

In [24]:
np.shape(train_X)

(250000, 30)

In [26]:
norm_train_X = np.copy(train_X)
norm_test_X = np.copy(test_X)

for i in range(0,(len(header)-1)):
    norm_train_X[:,i] /= np.mean(np.abs(train_X[:,i]),axis=0)
    norm_test_X[:,i] /= np.mean(np.abs(test_X[:,i]),axis=0)

In [27]:
predProb,pred = logisticReg(
    norm_train_X,train_labels,
    norm_test_X,
    test_labels)[0:2];
fullEvaluation(predProb,pred,test_weights,test_labels);

Signals in test-data: 187708.0
Comparison of [o]riginal and [a]djusted predictions:
 - [o]Signals read: 150363.0 
 - [a]Signals read: 203480 
 -- Difference: -53117.0 
 - [o]Correct labels: 412811 | signals: 100441 | background: 312370 
 ----- wrong signals: 87267 | background: 49922 
 - [a]Correct labels: 411582 | signals: 126385 | background: 285197 
 ----- wrong signals: 61323 | background: 77095 
 -- Difference labels: -1229 | signals: 25944 | background: -27173 
 - [o]AMS: 2.848508104990537 
 - [a]AMS: 2.903120098938839 (threshold = 0.414141414141 )
 -- Difference: -0.05461199394830185 



In [28]:
tb.createSolutionFile(test_eventList,predProb[:,1],0.41,"F:\BA_git\Data\Solutions\solution_logReg_test_normed.csv")

In [ ]:
featList = ['DER_mass_MMC','DER_mass_transverse_met_lep','DER_mass_vis']
(new_header,new_test_data,new_test_weights,new_test_labels) = kD.getCustomDataSet(featList,kSet = "v")
(new_header,new_train_data,new_train_weights,new_train_labels) = kD.getCustomDataSet(featList,kSet = "t")
predProb,pred = logisticReg(
    new_train_data,new_train_labels,
    new_test_data, new_test_labels);
fullEvaluation(predProb,pred,new_test_weights,new_test_labels);

In [ ]:
featList = kD.getFeatureListNoErrors()
(new_header,new_test_data,new_test_weights,new_test_labels) = kD.getCustomDataSet(featList,kSet = 'b')
(new_header,new_train_data,new_train_weights,new_train_labels) = kD.getCustomDataSet(featList,kSet = "t")

In [ ]:
test_X = new_test_data[:,1:]
test_weights = new_test_weights
test_labels = new_test_labels

train_X = new_train_data[:,1:]
train_labels = new_train_labels

In [ ]:
predProb,pred = logisticReg(
    train_X,train_labels,
    test_X, test_labels);
fullEvaluation(predProb,pred,new_test_weights,new_test_labels);

In [ ]:
featList = ["DER_mass_MMC",
            "DER_mass_transverse_met_lep",
            "DER_deltaeta_jet_jet",
            "DER_met_phi_centrality",
            "DER_pt_ratio_lep_tau"]
(new_header,new_test_data,new_test_weights,new_test_labels) = kD.getCustomDataSet(featList,kSet = 'v')
(new_header,new_train_data,new_train_weights,new_train_labels) = kD.getCustomDataSet(featList,kSet = "t")

In [ ]:
test_X = new_test_data[:,1:]
test_weights = new_test_weights
test_labels = new_test_labels

train_X = new_train_data[:,1:]
train_labels = new_train_labels

In [ ]:
predProb,pred = logisticReg(
    train_X,train_labels,
    test_X, test_labels);
fullEvaluation(predProb,pred,new_test_weights,new_test_labels);